In [ ]:
import cv2
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
from IPython.display import display
import threading
import time
import os

from llm_infer import llm_infer

# 创建一个摄像头捕获对象
cap = cv2.VideoCapture(0)

# 创建图像显示区域和按钮
image_widget = widgets.Image(format='jpeg', width=400, height=300)
captured_image_widget = widgets.Image(format='jpeg', width=400, height=300)
button_capture = widgets.Button(description="采集图像")
button_see = widgets.Button(description="你看到什么")
html_text = """
    请你根据你的理解，基于python只实现编码plan()函数，将桌面上所有的物体移动到目标位置，输出代码和描述
    以下是你能够直接调用的API:<br>
    1. <b>get_location_by_name(name)</b>: 根据输入的name获取其对应物体所在的xyz位置，用来找到对应物体的位置<br>
    2. <b>move_tool(xyz)</b>: 将末端夹爪移动到xyz位置<br>
    3. <b>grasp()</b>: 末端夹爪执行抓取，能够抓住当前机器人夹爪附近的物体<br>
    4. <b>ungrasp()</b>: 松开末端夹爪抓取，所夹住的物体会落在当前位置<br>
    5. <b>get_names_on_table()</b>: 返回类型List，返回所有在桌面上物体的名字。<br>
    6. <b>get_boxposition()</b>: 返回可以暂时存放物体的box的xyz位置
"""
normal_text = """
    你必须如下所有API:
    1. get_location_by_name(name): 根据输入的name获取其对应物体所在的xyz位置，用来找到对应物体的位置
    2. move_tool(xyz): 将末端夹爪移动到xyz位置
    3. grasp(): 末端夹爪执行抓取，能够抓住当前机器人夹爪附近的物体
    4. ungrasp(): 松开末端夹爪抓取，所夹住的物体会落在当前位置
    5. get_names_on_table(): 返回类型List，返回所有在桌面上物体的名字。
    6. get_boxposition(): 返回可以暂时存放物体的box的xyz位置
    请你根据你的理解，基于python实现简短的plan()函数，将你看到的所有物体移动到目标位置，输出英文代码不要描述，必须简短
"""
text_block = widgets.HTML(html_text)
button_do = widgets.Button(description="你能做什么")
output_text = widgets.Output()

# 显示控件
display(image_widget, button_capture, captured_image_widget, button_see, text_block, button_do, output_text)

# 全局变量，用于存储采集的图像
captured_image = None
running = True

# 实时更新摄像头图像的回调函数
def update_camera_image():
    while running:
        ret, frame = cap.read()
        if ret:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)
            buffer = BytesIO()
            pil_img.save(buffer, format="JPEG")
            image_widget.value = buffer.getvalue()
        time.sleep(0.1)  # 控制帧率

# 按钮点击事件，采集当前图像并保存
def capture_image(b):
    global captured_image
    ret, frame = cap.read()
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        captured_image = Image.fromarray(frame_rgb)

        # 将图片显示在 captured_image_widget 中
        buffer = BytesIO()
        captured_image.save(buffer, format="JPEG")
        captured_image_widget.value = buffer.getvalue()

        # 保存采集的图像到本地
        save_directory = "captured_images"
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)  # 创建目录
        image_path = os.path.join(save_directory, "captured_image.jpg")
        captured_image.save(image_path)
        print(f"图像已保存到: {image_path}")

# 设置按钮点击事件
button_capture.on_click(capture_image)

# 定义异步推理的函数
def infer_async(text, button_description):
    result = llm_infer(text)
    with output_text:
        output_text.clear_output()
        # print(f"{button_description}: {result}")

# 定义“what you see in the picture”的推理事件
def infer_see(b):
    threading.Thread(target=infer_async, args=("请输出桌上所有物体", "Answer:")).start()

# 定义“what you can do in this picture”的推理事件
def infer_do(b):
    threading.Thread(target=infer_async, args=(normal_text, "What you can do")).start()

# 为按钮分配事件
button_see.on_click(infer_see)
button_do.on_click(infer_do)

# 开启一个线程用于实时更新摄像头图像
camera_thread = threading.Thread(target=update_camera_image)
camera_thread.start()

# 当结束时释放摄像头资源
def stop_camera():
    global running
    running = False
    camera_thread.join()
    cap.release()

# 确保程序结束时摄像头释放
import atexit
atexit.register(stop_camera)
